# **Modélisation 1**

À partir de l'article Medium référencé dans le README, nous allons essayer de créer un algorithme de recommendation à partir de notre base. Au vu du contenu de notre base, nous n'allons pas vraimet pouvoir implementer une partie en fonction de ce que les utilisateurs ayant aimé un film auraient aussi aimé, et plus nous concentrer sur les caractéristiques des films.

In [205]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [206]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

def fonction_charge(path) :
    df = pd.read_csv(path)
    df_f = df.drop_duplicates()
    df_f['release_date'] = pd.to_datetime(df_f['release_date'])
    # De la même façon on convertit la colonne 'budget' qui est composée de chaînes de caractères et non de valeurs numériques
    df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
    df_f['genres_list'] = df_f['genres_list'].apply(ast.literal_eval)
    df_f['countries_prod'] = df_f['countries_prod'].apply(ast.literal_eval)
    df_f['languages_list'] = df_f['languages_list'].apply(ast.literal_eval)
    df_f['prod_companies'] = df_f['prod_companies'].apply(ast.literal_eval)
    df_f['directors'] = df_f['directors'].apply(safe_literal_eval)
    return df_f 

In [207]:
df = fonction_charge('../cleaning_data/Final_database.csv')
df.columns

/tmp/ipykernel_6813/1525667594.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['release_date'] = pd.to_datetime(df_f['release_date'])
/tmp/ipykernel_6813/1525667594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
/tmp/ipykernel_6813/1525667594.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

Index(['adult', 'budget', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'release_date', 'revenue',
       'runtime', 'tagline', 'title', 'vote_average', 'vote_count',
       'directors', 'collection_name', 'genres_list', 'countries_prod',
       'languages_list', 'prod_companies', 'Award_mains', 'Mains_cat',
       'Award_fest', 'Fest_cat'],
      dtype='object')

In [208]:
movies_df = df
movies_df['genres_ex'] = movies_df['genres_list'].apply(lambda x: ', '.join(map(str, x)))
movies_df['direct_ex'] = movies_df['directors'].apply(lambda x: ', '.join(map(str, x)))
movies_df
# Create a new column with combined information
movies_df['combined_info'] = (
    movies_df['overview'] + ' ' + movies_df['genres_ex'] + ' ' +  movies_df['direct_ex'] + ' ' +  movies_df['collection_name'].fillna('')
)
movies_df['combined_info']


0                                                                                                                                                                                                               An ex-special forces operative takes a job to provide security for a journalist as she interviews a dictator, but a military coup breaks out in the middle of the interview, they are forced to escape into the jungle where they must survive. Action, Comedy Pierre Morel 
1                                                                                                                                                                                                                                                                                           When oil is discovered in 1920s Oklahoma under Osage Nation land, the Osage people are murdered one by one—until the FBI steps in to unravel the mystery. Crime, Drama, History Martin Scorsese 
2                                             

In [209]:
movies_df['combined_info'].fillna('', inplace=True)

# initialisation et tfidf
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['combined_info'])


In [210]:
# la matrice de similarité
cos_mat = linear_kernel(tfidf_matrix, tfidf_matrix)

In [211]:
diag = 0
for i in range(len(cos_mat)):
    diag+= cos_mat[i][i]
    
print(diag)

9158.0


In [212]:
def get_recommendations(movie, n):
    
    # récupérer l'index
    index = movies_df[movies_df['title']== movie].index[0]
    
    # trier les n films plus proches    
    similar_movies = sorted(list(enumerate(cos_mat[index])), reverse=True, key=lambda x: x[1]) 
    
    # extraire les noms et les afficher
    recomm = []
    for i in similar_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm


In [213]:
get_recommendations("Leo", 5)

['The Week Of',
 'Cat Person',
 'Notes on a Scandal',
 'Life in a Year',
 'Home Alone 2: Lost in New York']

In [214]:
get_recommendations("Everything Everywhere All at Once", 4)

['Suck Me Shakespeer', 'Ballerina', 'Obsessed', 'Hyungsu: Forbidden Love']

In [215]:
cos_mat = linear_kernel(tfidf_matrix, tfidf_matrix)

print(tfidf_matrix.shape ,
cos_mat.shape)
#vérification de la dimension pour les opérations ensuite 

(9180, 30824) (9180, 9180)


In [216]:
def get_keywords_recommendations(keywords, n):
    
    keywords = keywords.split()
    keywords = " ".join(keywords)
    
    # transformer le str en vect
    key_tfidf = tfidf.transform([keywords]) 
    
    # calcul de la cosine similarity - qq soucis avec la dimension reglés
   
    result = cosine_similarity(key_tfidf, tfidf_matrix)

    
    # trier les n meilleurs   
    similar_key_movies = sorted(list(enumerate(result[0])), reverse=True, key=lambda x: x[1])
    
    # extraction des noms et retour des noms 
    recomm = []
    for i in similar_key_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm

In [217]:
get_keywords_recommendations("Christopher Nolan Magic", 4)

['Oppenheimer', 'The Prestige', 'Tenet', 'Interstellar']

In [218]:
get_keywords_recommendations('Comedy about crazy children', 3)

['Children of the Corn IV: The Gathering',
 'Bedknobs and Broomsticks',
 'Funny Thing About Love']

In [219]:
get_keywords_recommendations('Oscar', 3)

['Enter the Void',
 'Oscar and the Lady in Pink',
 'Big Trip 2: Special Delivery']

In [220]:
#Nous pouvons aussi essayer d'intégrer les nominations car nous les avons stockées sous forme de str 
movies_df['Mains_cat'] = movies_df['Mains_cat'].fillna(" ")
movies_df['Fest_cat'] = movies_df['Fest_cat'].fillna(" ")
movies_df['mains_ex'] = movies_df['Mains_cat'].apply(lambda x: ''.join(map(str, x)))
movies_df['fest_ex'] = movies_df['Fest_cat'].apply(lambda x: ''.join(map(str, x)))
# Create a new column with combined information
movies_df['combined_info_2'] = (
    movies_df['combined_info']+ ' ' +  movies_df['fest_ex'] + ' ' + movies_df['mains_ex'].fillna('')
)



In [221]:
movies_df['combined_info_2'].fillna('', inplace=True)

# initialisation et tfidf
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_2 = tfidf.fit_transform(movies_df['combined_info_2'])
# la matrice de similarité
cos_mat_2 = linear_kernel(tfidf_matrix_2, tfidf_matrix_2)

In [222]:
#vérification 
diag = 0
for i in range(len(cos_mat)):
    diag+= cos_mat_2[i][i]
    
print(diag)

9158.0


In [223]:
def get_recommendations_aw(movie, n):
    
    # récupérer l'index
    index = movies_df[movies_df['title']== movie].index[0]
    
    # trier les n films plus proches    
    similar_movies = sorted(list(enumerate(cos_mat_2[index])), reverse=True, key=lambda x: x[1]) 
    
    # extraire les noms et les afficher
    recomm = []
    for i in similar_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm


In [224]:
get_recommendations_aw("Parasite", 10)

['Diary of a Wimpy Kid: Rodrick Rules',
 'Diary of a Wimpy Kid: Rodrick Rules',
 'Diary of a Wimpy Kid',
 'Diary of a Wimpy Kid: Dog Days',
 'Diary of a Wimpy Kid: The Long Haul',
 'Diary of a Wimpy Kid',
 'The Snowman',
 'The Addams Family',
 'Dealing with Christmas',
 'Pixels']

In [225]:
get_recommendations("Parasite", 10)

['Diary of a Wimpy Kid: Rodrick Rules',
 'Diary of a Wimpy Kid: Rodrick Rules',
 'Diary of a Wimpy Kid',
 'Diary of a Wimpy Kid: Dog Days',
 'Diary of a Wimpy Kid: The Long Haul',
 'Diary of a Wimpy Kid',
 'The Snowman',
 'The Addams Family',
 'Dealing with Christmas',
 'Pixels']

L'intégration des recompenses ne semble pas avoir eu un effet escompté car la prédiction pour Parasite de Bong Joon-ho ne semble pas avoir changé malgré l'ajout des récompenses dans le texte. Essayons pour le keyword.

In [226]:
def get_keywords_recommendations_aw(keywords, n):
    
    keywords = keywords.split()
    keywords = " ".join(keywords)
    
    # transformer le str en vect
    key_tfidf_2 = tfidf.transform([keywords]) 
    
    # calcul de la cosine similarity - qq soucis avec la dimension reglés
   
    result = cosine_similarity(key_tfidf_2, tfidf_matrix_2)

    
    # trier les n meilleurs   
    similar_key_movies = sorted(list(enumerate(result[0])), reverse=True, key=lambda x: x[1])
    
    # extraction des noms et retour des noms 
    recomm = []
    for i in similar_key_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm

In [231]:
get_keywords_recommendations_aw('Oscar', 3)

['West Side Story', 'A Place in the Sun', 'Stagecoach']

Le résultat change en rentrant le nom de la récompense elle-même. Le problème est peut-être lié au passage entre l'anglais et le français.

# Améliorations de l'algorithme possibles 


Bien que satisfaisant pour ce que nous souhaitions faire, notre algorithme aurait besoin d'un certain nombre d'améliorations pour être réellement intéressant (hors du contexte d'un projet de programmation). D'un côté, la dimension "Item-Item" pourrait être améliorée avec l'ajout de nouvelles informations, comme les acteurs, ou les producteurs par exemple. Aussi, nous aurions pu essayé d'intégrer certaines variables numériques que nous avions récupérer mais que nous n'avons pas pu exploiter (par exemple la note moyenne, mais aussi le budget ou la durée par exemple). D'un autre, nous pourrions essayer une approche plus "User-User" en essayant de récolter des informations du côté des utilisateurs. Mais même sans rentrer dans une logique purement "User-User", récolter plus de donnée en rapport aux utilisateurs pourrait aider à affiner cet algorithme. Enfin, bien sûr, une des améliorations les plus évidentes serait d'enrichir la base avec de nouveaux films 